In [1]:
import time, glob, os
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as utils
from PIL import Image
from skimage import io
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
class DeepStar(nn.Module):
    def __init__(self):
        super(DeepStar, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.pool1 = torch.nn.MaxPool2d(kernel_size=4, stride=4, padding=0)
        
        self.conv2 = torch.nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1)
        self.pool2 = torch.nn.MaxPool2d(kernel_size=4, stride=4, padding=0)
        
        self.fc1 = torch.nn.Linear(32 * 16 * 16, 64)
        self.fc2 = torch.nn.Linear(64, 2)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = x.view(-1, 32 * 16 *16)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    
net = DeepStar()

In [3]:
class PathDataLoader(utils.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = glob.glob(data_dir + "*.png")
        self.to_tensor = transforms.ToTensor()
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        file_path = self.files[idx]
        
        image_name = os.path.basename(file_path)
        img_as_img = Image.open(file_path)
        img_as_tensor = self.to_tensor(img_as_img)
        
        numbers = image_name.split("_")
        #print(len(numbers))
        
        x = float(numbers[0]) / 256.0
        y = float(numbers[1]) / 256.0
        
        return (img_as_tensor, [x, y])

In [4]:
def createLossAndOptimizer(net, learning_rate=0.001):
    loss = torch.nn.L1Loss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [5]:
train_data = PathDataLoader("data/heightmap/images/")
train_loader = utils.DataLoader(train_data, batch_size=5, shuffle=True, num_workers=4)

val_data = PathDataLoader("data/heightmap/validation/")
val_loader = utils.DataLoader(train_data, batch_size=5, shuffle=True, num_workers=4)

In [6]:
batch_size = 20
n_epochs = 20
learning_rate = 0.002

n_batches = len(train_loader)

loss, optimizer = createLossAndOptimizer(net, learning_rate)

training_start_time = time.time()

#Loop for n_epochs
print("Training with {} images".format(len(train_loader)))
for epoch in range(n_epochs):

    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0

    for i, data in enumerate(train_loader, 0):
        img, pos = data
        
        optimizer.zero_grad()

        expected = torch.ones(len(img),2,requires_grad=False)
        for a in range(len(img)):
            expected[a][0] = pos[0][a]
            expected[a][1] = pos[1][a]
        
        output = net(img)
        loss_size = loss(output, expected)
        loss_size.backward()
        optimizer.step()
        
        running_loss += loss_size.data.item()
        total_train_loss += loss_size.data.item()
        
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
            #Reset running loss and time
            running_loss = 0.0
            start_time = time.time()
            
    total_val_loss = 0
    for inputs, labels in val_loader:

        #Wrap tensors in Variables
        img, expected = Variable(img), Variable(expected)

        #Forward pass
        val_outputs = net(img)
        val_loss_size = loss(val_outputs, expected)
        total_val_loss += val_loss_size.data.item()

    print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))

print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

Training with 202 images
Epoch 1, 10% 	 train_loss: 0.27 took: 8.59s
Epoch 1, 20% 	 train_loss: 0.21 took: 8.35s
Epoch 1, 31% 	 train_loss: 0.22 took: 8.34s
Epoch 1, 41% 	 train_loss: 0.21 took: 8.80s
Epoch 1, 51% 	 train_loss: 0.21 took: 8.84s
Epoch 1, 62% 	 train_loss: 0.20 took: 8.53s
Epoch 1, 72% 	 train_loss: 0.18 took: 8.38s
Epoch 1, 83% 	 train_loss: 0.19 took: 7.75s
Epoch 1, 93% 	 train_loss: 0.18 took: 7.72s
Validation loss = 0.18
Epoch 2, 10% 	 train_loss: 0.19 took: 8.20s
Epoch 2, 20% 	 train_loss: 0.20 took: 8.55s
Epoch 2, 31% 	 train_loss: 0.20 took: 8.79s
Epoch 2, 41% 	 train_loss: 0.19 took: 8.07s
Epoch 2, 51% 	 train_loss: 0.19 took: 8.50s
Epoch 2, 62% 	 train_loss: 0.22 took: 8.23s
Epoch 2, 72% 	 train_loss: 0.20 took: 8.00s
Epoch 2, 83% 	 train_loss: 0.19 took: 8.14s
Epoch 2, 93% 	 train_loss: 0.16 took: 8.08s
Validation loss = 0.12
Epoch 3, 10% 	 train_loss: 0.12 took: 7.87s
Epoch 3, 20% 	 train_loss: 0.11 took: 8.07s
Epoch 3, 31% 	 train_loss: 0.10 took: 10.77s
Epoc

Epoch 20, 41% 	 train_loss: 0.03 took: 7.73s
Epoch 20, 51% 	 train_loss: 0.04 took: 8.27s
Epoch 20, 62% 	 train_loss: 0.03 took: 7.92s
Epoch 20, 72% 	 train_loss: 0.04 took: 7.74s
Epoch 20, 83% 	 train_loss: 0.04 took: 9.22s
Epoch 20, 93% 	 train_loss: 0.04 took: 8.44s
Validation loss = 0.03
Training finished, took 2440.60s


In [30]:
point = (207, 197)
img = Image.open("data/{0}_{1}_confirm.png".format(point[0], point[1]))
to_tensor = transforms.ToTensor()
img_tensor = to_tensor(img).unsqueeze(0)

output = net(img_tensor)
print(output)
print(round(output[0][0].item() * 256), round(output[0][1].item() * 256))

tensor([[0.7987, 0.7604]], grad_fn=<AddmmBackward>)
204 195


In [ ]:
start = (34, 153)
end = (231, 123)

img = Image.open("data/map.png").format(point[0], paint[1])